In [1]:
pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.5 MB ? eta -:--:--
   -------------- ------------------------- 0.5/1.5 MB 2.1 MB/s eta 0:00:01
   ------------------------------------ --- 1.3/1.5 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 2.3 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
df = pd.read_csv(r"C:\Users\82103\Downloads\vip_customers_all.csv")

C:\Users\82103\AppData\Local\Temp\ipykernel_23756\3409864554.py:2: DtypeWarning: Columns (339) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\82103\Downloads\vip_customers_all.csv")


In [ ]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score

# 1. 데이터 로드 (가정)
# df = pd.read_csv('vip_customers_all.csv')
# (여기서 df에 churn_target, churn_score 컬럼이 이미 만들어져 있다고 가정합니다!)

# ============================================================================
# 핵심 1: 불필요한 컬럼 정의
# - 타겟변수(정답): churn_target, churn_score
# - 식별자/날짜: 발급회원번호, 기준년월 (모델이 외우면 안 됨)
# ============================================================================
target_col = 'churn_target'  # 정답지
drop_cols = ['churn_target', 'churn_score', '발급회원번호', '기준년월']

# ============================================================================
# 핵심 2: 월별로 데이터 쪼개기 (Time Split)
# ============================================================================
# 1) 학습 데이터 (7, 8, 9, 10월)
train_df = df[df['기준년월'].isin([201807, 201808, 201809, 201810])]

# 2) 검증 데이터 (11월) - 모델 채점용
valid_df = df[df['기준년월'] == 201811]

# 3) 실전 예측 데이터 (12월) - 정답(Target)이 없는 미래 데이터
predict_df = df[df['기준년월'] == 201812]

# ============================================================================
# 핵심 3: X(문제), y(정답) 나누기
# ============================================================================
# 학습용
X_train = train_df.drop(columns=drop_cols)
y_train = train_df[target_col]

# 검증용
X_valid = valid_df.drop(columns=drop_cols)
y_valid = valid_df[target_col]

# 실전 예측용 (정답 y가 없음)
X_predict = predict_df.drop(columns=drop_cols)
predict_ids = predict_df['발급회원번호'] # 결과 매칭용 ID 킵해두기

# 문자열 -> category 변환 (LightGBM용)
for col in X_train.columns:
    if X_train[col].dtype == 'object':
        X_train[col] = X_train[col].astype('category')
        X_valid[col] = X_valid[col].astype('category')
        X_predict[col] = X_predict[col].astype('category')

# ============================================================================
# 핵심 4: 모델 학습
# ============================================================================
model = lgb.LGBMClassifier(n_estimators=1000, random_state=42)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)], # 11월 데이터로 시험 보면서 학습
    eval_metric='logloss',
    callbacks=[lgb.early_stopping(stopping_rounds=50)] # 더 이상 안 똑똑해지면 멈춤
)

# ============================================================================
# 핵심 5: 12월 고객에 대한 '내년 1월 이탈 확률' 예측
# ============================================================================
pred_prob = model.predict_proba(X_predict)[:, 1]

# 결과 정리
final_result = pd.DataFrame({
    '발급회원번호': predict_ids,
    '이탈예측확률': pred_prob
})

print(final_result.sort_values('이탈예측확률', ascending=False).head(10))

In [ ]:
pred_valid = model.predict(X_valid)
print("검증(11월) 정확도:", accuracy_score(y_valid, pred_valid))